In [5]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

In [6]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

In [8]:
no_of_classes = 10
input_shape = (28, 28, 1)

noise_factor=.5
x_train_noise=x_train+noise_factor*np.random.normal(loc=0,scale=1.0,size=x_train.shape)
x_test_noise=x_test+noise_factor*np.random.normal(loc=0,scale=1.0,size=x_test.shape)
x_train_noise=np.clip(x_train_noise,0.,1.)
x_test_noise=np.clip(x_test_noise,0.,1.)

x_train_noise = np.expand_dims(x_train_noise, -1) # Make sure images have shape (28, 28, 1)
x_test_noise = np.expand_dims(x_test_noise, -1)

y_train = keras.utils.to_categorical(y_train, no_of_classes)
y_test = keras.utils.to_categorical(y_test, no_of_classes)

In [9]:
#add encoder
Input_img = layers.Input(shape=(28, 28, 1))  
x1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(Input_img)
x2 = layers.MaxPooling2D((2, 2), padding='same')(x1)
x3 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x2)
encoded = layers.MaxPooling2D((2, 2), padding='same')(x3)

#add decoder
x3 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(encoded)
x2 = layers.UpSampling2D((2, 2))(x3)
x1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x2)
x = layers.UpSampling2D((2, 2))(x1)
decoded = layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(Input_img, decoded)
autoencoder.compile(optimizer="adam", loss='binary_crossentropy')

In [10]:
autoencoder.fit(x_train_noise, x_train,
                epochs=20,
                batch_size=128)

Train on 60000 samples
Epoch 1/20
60000/60000 [==============================] - 131s 2ms/sample - loss: 0.1600
Epoch 2/20
60000/60000 [==============================] - 131s 2ms/sample - loss: 0.1107
Epoch 3/20
60000/60000 [==============================] - 129s 2ms/sample - loss: 0.1056
Epoch 4/20
60000/60000 [==============================] - 152s 3ms/sample - loss: 0.1032
Epoch 5/20
60000/60000 [==============================] - 144s 2ms/sample - loss: 0.1015
Epoch 6/20
60000/60000 [==============================] - 141s 2ms/sample - loss: 0.1002
Epoch 7/20
60000/60000 [==============================] - 134s 2ms/sample - loss: 0.0993
Epoch 8/20
60000/60000 [==============================] - 135s 2ms/sample - loss: 0.0985
Epoch 9/20
60000/60000 [==============================] - 136s 2ms/sample - loss: 0.0979
Epoch 10/20
60000/60000 [==============================] - 131s 2ms/sample - loss: 0.0974
Epoch 11/20
60000/60000 [==============================] - 128s 2ms/sample - loss: 0.0

In [13]:
#predict
predicted=autoencoder.predict(x_test_noise)

#denoised input
train_renoised = autoencoder.predict(x_train_noise)
test_renoised =autoencoder.predict(x_test_noise)

#build classifer model
model = keras.Sequential()
model.add(keras.Input(shape=input_shape))
model.add(layers.Conv2D(32, kernel_size=(3, 3), activation="relu"))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Conv2D(128, kernel_size=(3, 3), activation="relu"))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(no_of_classes, activation="softmax"))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 11, 11, 128)       36992     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 5, 5, 128)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 3200)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 3200)              0         
_________________________________________________________________
dense (Dense)                (None, 10)               

In [14]:
batch_size = 128
epochs = 10
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.fit(train_renoised, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Train on 54000 samples, validate on 6000 samples
Epoch 1/10
54000/54000 [==============================] - 31s 573us/sample - loss: 0.3718 - accuracy: 0.8866 - val_loss: 0.1434 - val_accuracy: 0.9570
Epoch 2/10
54000/54000 [==============================] - 39s 726us/sample - loss: 0.1652 - accuracy: 0.9479 - val_loss: 0.1126 - val_accuracy: 0.9652
Epoch 3/10
54000/54000 [==============================] - 33s 617us/sample - loss: 0.1401 - accuracy: 0.9555 - val_loss: 0.1010 - val_accuracy: 0.9698
Epoch 4/10
54000/54000 [==============================] - 35s 647us/sample - loss: 0.1265 - accuracy: 0.9595 - val_loss: 0.0931 - val_accuracy: 0.9715
Epoch 5/10
54000/54000 [==============================] - 35s 656us/sample - loss: 0.1174 - accuracy: 0.9612 - val_loss: 0.0913 - val_accuracy: 0.9712
Epoch 6/10
54000/54000 [==============================] - 35s 639us/sample - loss: 0.1109 - accuracy: 0.9635 - val_loss: 0.0868 - val_accuracy: 0.9730
Epoch 7/10
54000/54000 [=====================

In [15]:
#Evaluate
score = model.evaluate(test_renoised, y_test)
print("Test loss:", score[0])
print("Test accuracy:", score[1])
#observing a lower accuracy might need to try more epochs or with other configurations

10000/10000 [==============================] - 2s 203us/sample - loss: 0.0814 - accuracy: 0.9721
Test loss: 0.08139649834744632
Test accuracy: 0.9721
